# Transformers AutoClass 

### Tokenizer, Model Loading
- Huggingface 모델 허브에서 제공하는 처리 모델을 다운받아 로딩한다.
- 다운로드된 모델은 `사용자 home 디렉토리\.cache\huggingface` 에 저장된다.
- 미리 학습된 언어 모델을 다운받아 사용할 때는 그 언어모델이 사용한 tokenizer를 같이 받아서 사용한다.

### [Auto Classes](https://huggingface.co/docs/transformers/model_doc/auto)
- Huggingface 에서 제공하는 다양한 모델들은 손쉽게 불러오고 사용할 수 있도록 설계된 유틸리티 클래스들을 말한다.
- 미리 학습된 특정 모델의 이름(모델 허브상에서 제공되는 이름)이나 저장된 local 경로를 제공하면 해당 모델에 맞는 적절한 클래스와 구성 요소를 자동으로 로드한다.
- 사용자는 모델을 사용하기 위한 정확한 클래스를 몰라도 쉽게 다양한 종류의 모델을 사용할 수있다.

#### 주요 Auto Class
- 기본 모델 Loading
    1. **AutoModel**
       - 주어진 모델 이름에 맞는 사전 학습된 모델을 자동으로 로드한다.
       - 예: `AutoModel.from_pretrained("bert-base-uncased")`: BERT 모델을 로드한다.
    2. **AutoTokenizer**
       - 해당 모델에 적합한 토크나이저를 자동으로 로드한다.
       - 예: `AutoTokenizer.from_pretrained("bert-base-uncased")`: BERT 모델에 맞는 토크나이저를 로드한다.
    3. **AutoConfig**
       - 모델의 설정(config)을 자동으로 로드한다. 모델 설정에는 모델의 하이퍼파라미터와 모델 구조 정보가 포함된다. 이 설정을 이용해 모델을 생성할 수있다.
       - 예: `AutoConfig.from_pretrained("bert-base-uncased")`
- Task 처리 모델 Loading
    - Pretrained backbone 모델에 각 task 에 맞는 estimator layer를 추가한 모델을 생성해 제공한다.
    - 주요 모델들
        1. **AutoModelForSequenceClassification**
           - 시퀀스(Text) 분류 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")`
        2. **AutoModelForQuestionAnswering**
           - 질문-응답 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")`
        3. **AutoModelForTokenClassification**
           - 토큰 분류 작업(예: 개체명 인식)을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForTokenClassification.from_pretrained("bert-base-uncased")`

## kcbert
- BERT 모델을 한글 텍스트로 학습 시킨 Pretrained model.
    - BERT는 Transformer의 Encoder 부분을 이용해 구현된 언어모델
    - https://arxiv.org/abs/1810.04805 
- https://huggingface.co/beomi/kcbert-base

### 토크나이저, 모델 로드

In [2]:
from transformers import AutoModel, AutoTokenizer

model_id = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)
type(tokenizer), type(model)

(transformers.models.bert.tokenization_bert_fast.BertTokenizerFast,
 transformers.models.bert.modeling_bert.BertModel)

In [3]:
# 단일 문서 입력: padding, truncaton (토큰 수 맞추는 작업)
tokens = tokenizer(
    "나는 어제 친구와 밥을 먹었다.",
    return_tensors="pt"
)
tokens

{'input_ids': tensor([[    2,  8616,  9909,  9025,  4196, 23905, 21452,  4020,    17,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
tokenizer.encode("나는 어제 친구와 밥을 먹었다.") # input_id만 출력
tokenizer.decode([2, 8616, 9909, 9025, 4196, 23905, 21452, 4020, 17, 3]) # token들 -> 문장
tokenizer.decode(
    [2, 8616, 9909, 9025, 4196, 23905, 21452, 4020, 17, 3],
    skip_special_tokens=True # 스페셜 토큰은 제거
)


'나는 어제 친구와 밥을 먹었다.'

In [12]:
# 개별 토큰 관련 조회
tokenizer.convert_ids_to_tokens(8616) # 토큰 id -> 토큰 문자열
tokenizer.convert_ids_to_tokens([2, 8616, 9909])

tokenizer.convert_tokens_to_ids("어제")
tokenizer.convert_tokens_to_ids(["어제", "너는", "밖"])

[9909, 9039, 1482]

### 입력값 토큰화

In [37]:
# 토크나이저 전체 어휘수
print(tokenizer.vocab_size)
tokenizer.get_vocab() # 어휘사전 조회

30000


{'바른미래': 18303,
 '증오': 16511,
 '양육': 20997,
 '##면피': 13843,
 '23': 15811,
 '##갗': 6530,
 '언플': 15602,
 '##os': 28528,
 '바다': 12965,
 '남발': 17945,
 '비웃': 13716,
 '여자': 8275,
 'IT': 26525,
 '자국민이': 24480,
 '우방': 13957,
 '감정을': 23603,
 '##놌': 6524,
 '맨날': 8939,
 '입국금지': 9564,
 '뿌리': 11807,
 '##권도': 17856,
 '어디든': 25410,
 '와이': 12597,
 '##✊': 5735,
 '##병': 4619,
 '##🤩': 6232,
 '무관심': 29634,
 '##™': 7159,
 '미쳤냐': 25701,
 '지구': 10074,
 '스펙': 14643,
 '심어': 21446,
 '개웃': 16623,
 '##가루': 17849,
 '백': 1505,
 '굉장히': 17611,
 '일벌': 28524,
 '##하겠지만': 16162,
 '대다수가': 19444,
 '8월': 25794,
 '꼬라지': 9648,
 '뺏어': 23479,
 '셌': 1872,
 '##섘': 6034,
 '##감이다': 15212,
 '양심없는': 24188,
 '##🤺': 7036,
 '##🎭': 7883,
 '현정권이': 27435,
 '##민주주의': 9453,
 '치매': 9533,
 '🍉': 3604,
 '강제': 9117,
 '퉵': 3193,
 '##춘': 4658,
 '##날에': 24290,
 '어디갔냐': 23458,
 '##식당': 26700,
 '##숵': 6072,
 '파리': 23008,
 '7년': 25148,
 '하니깐': 23942,
 '당신들은': 15297,
 '수단과': 29136,
 '##숳': 7593,
 '##존': 4427,
 '🇨': 3548,
 '##은이': 8697,
 '##연이': 18445,
 

In [ ]:
##################
# 여러 문서 입력
##################

In [13]:
sentences = [
    "안녕",
    "Hugging Face는 인공지능(AI)과 자연어 처리(NLP) 분야에서 혁신적인 도구와 모델을 제공하는 AI 스타트업이다.",
    "2016년에 설립된 이 회사는 주로 오픈소스 라이브러리와 사전 학습된 NLP 모델을 제공을 제공한다."
]

In [19]:
sent_tokens = tokenizer(
    sentences
    # return_tensors="pt"
)

In [22]:
print(len(sent_tokens['input_ids']), sent_tokens['input_ids'])
print(len(sent_tokens['attention_mask']))
print(len(sent_tokens['token_type_ids']))

3 [[2, 19017, 3], [2, 41, 4223, 4403, 4403, 18940, 39, 4243, 4773, 4226, 4008, 14583, 25061, 11, 22502, 12, 321, 10459, 4071, 9810, 11, 47, 4450, 4579, 12, 16029, 7971, 13064, 8097, 867, 4228, 4196, 16505, 4027, 13248, 7966, 22502, 12296, 4104, 22192, 4020, 17, 3], [2, 26182, 4113, 20684, 4130, 2451, 22088, 20002, 15999, 4266, 4103, 17564, 4408, 4053, 4038, 4196, 11202, 20323, 4130, 47, 4450, 4579, 16505, 4027, 13248, 4027, 13248, 8008, 17, 3]]
3
3


In [23]:
# 토큰수
for ids in sent_Tokens['input_ids']:
    print(len(ids))

3
43
30


In [ ]:
# max_length - 최대길이(토큰 수)를 지정
# truncation - 길이를 맞추기 위해서 토큰의 일부를 잘라내는 것
#            - True("longest"): 배치중에서 가장 긴 문서에 맞춘다. max_length 설정이 된 경우에는 max_length에 맞춘다
#            - False("do_not_truncate"): 기본값. Truncation을 안함
# padding - 길이를 맞추기 위해서 [PAD] 토큰으로 채우는 것
#         - True("longest"): 배치중에 가장 긴 문서에 맞춘다
#         - "max_length": max_length 설정에 맞춘다
#         - False("do_not_pad"): 기본값, Padding 처리 하지 않는다
# 순서 : truncation -> padding

In [35]:
sent_tokens2 = tokenizer(
    sentences,
    max_length=20,
    padding=True,
    truncation=True
)

In [30]:
for sent in sent_tokens2['input_ids']:
    print(len(sent))#, sent)

43
43
43


In [36]:
sent_tokens2['attention_mask'][0]

[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

### BERT 모델을 이용해 context vector 추출
#### Model 추론결과
- **last_hidden_state**
    - 모든 token들에 대한 feature
    - 출력이 **many**인 작업에 사용한다.
- **pooler_output**
    - 입력 문장, 텍스트에 대한 context vector 이다.
    - 이 값은 **문장을 입력받아 처리하는 task**(ex: 문서분류-감정분석,문장카테고리분류, 문장유사도 분석)의 입력으로 사용한다.

In [ ]:
tokens = tokenizer(
    sentences,
    max_length=20,
    padding=True,
    truncation=True
    return_tensors="pt"
)

In [ ]:
result = model(**tokens)

In [ ]:
result.last_hidden_state.shape
# [3:batch-문서개수, 20:seq_len(토큰수), 768]

In [ ]:
result.pooler_output.shape
# [3, 768]